In [1]:
from ekonlpy import Mecab
from ekonlpy.sentiment import MPCK
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [2]:
df = pd.read_csv("./data/minutes_df.csv")
call_df = pd.read_csv("./data/call_rate_diff.csv")
df

,date,title,contents
0,2009.05.12,금융통화위원회 의사록(2009년도 제12차)(2009.5.12),
1,2009.05.21,금융통화위원회 의사록(2009년도 제13차)(2009.5.21),
2,2009.06.11,금융통화위원회 의사록(2009년도 제14차)(2009.6.11),
3,2009.06.25,금융통화위원회 의사록(2009년도 제15차)(2009.6.25),
4,2009.07.09,금융통화위원회 의사록(2009년도 제16차)(2009.7.9),
...,...,...,...
295,2024.02.22,금융통화위원회 의사록(2024년도 제4차)(2024.2.22),일부 위원은 최근 금융·외환시장은 대체로 안정적인 흐름을 이어가고 있으며지난 통화정...
296,2024.03.14,금융통화위원회 의사록(2024년도 제5차)(2024.3.14),
297,2024.03.28,금융통화위원회 의사록(2024년도 제6차)(2024.3.28),
298,2024.04.12,금융통화위원회 의사록(2024년도 제7차)(2024.4.12),일부 위원은 미국 소비자물가(2024.3월) 발표 이후 일각에서 미 연준의 금리인상...


In [3]:
# contents가 없는 row 삭제
df["contents"] = df["contents"].apply(lambda x: x.strip())
df = df.replace("", np.nan)
df = df.dropna()
df

,date,title,contents
14,2010.05.12,금융통화위원회 의사록(2010년도 제10차)(2010.5.12),일부 위원은 남유럽사태 이후 각국이·｢ ｣금융기관에 대한 미시 거시 건전성감독의 중...
17,2010.07.09,금융통화위원회 의사록(2010년도 제14차)(2010.7.9),일부 위원은 금리 인상시 차익거래 유·｢ ｣인이 확대되어 외국인 채권투자자금 유입이...
19,2010.08.12,금융통화위원회 의사록(2010년도 제16차)(2010.8.12),일부 위원은 연동대출 취급확대가COFIX｢ ｣통화정책에 미치는 영향으로 금리파급경로...
25,2010.11.16,금융통화위원회 의사록(2010년도 제22차)(2010.11.16),일부 위원은 기준금리 인상이 내외금·｢ ｣리차 확대에 따른 재정거래 유인 증대를 초...
26,2010.12.09,금융통화위원회 의사록(2010년도 제24차)(2010.12.9),일부 위원은 월중 기준금리 인상에도 불11｢ ｣구하고 국고채 년물 금리가 사상 최저...
...,...,...,...
290,2023.04.11,금융통화위원회 의사록(2023년도 제7차)(2023.4.11),"일부 위원은 신협기구(상호금융, 신협, 새마을금고)의 부동산 관련 대출 비중이 과거..."
291,2024.05.23,금융통화위원회 의사록(2024년도 제10차)(2024.5.23),일부 위원은 유로지역 등 주요국이 미국에 앞서 선제적으로 정책금리를 인하할 가능성이...
292,2024.01.11,금융통화위원회 의사록(2024년도 제1차)(2024.1.11),일부 위원은 국내외 금융시장이 지난 통화정책방향 결정회의 이후 대체로 완화적이며 안...
295,2024.02.22,금융통화위원회 의사록(2024년도 제4차)(2024.2.22),일부 위원은 최근 금융·외환시장은 대체로 안정적인 흐름을 이어가고 있으며지난 통화정...


In [4]:
# 텍스트 전처리
mpck = MPCK()
# POS & tagging => ngrams
df["ex_ngrams"] = df["contents"].astype(str).apply(mpck.tokenize).apply(mpck.ngramize)
df

,date,title,contents,ex_ngrams
14,2010.05.12,금융통화위원회 의사록(2010년도 제10차)(2010.5.12),일부 위원은 남유럽사태 이후 각국이·｢ ｣금융기관에 대한 미시 거시 건전성감독의 중...,"[외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG, 장단기/NNG;금리/NN..."
17,2010.07.09,금융통화위원회 의사록(2010년도 제14차)(2010.7.9),일부 위원은 금리 인상시 차익거래 유·｢ ｣인이 확대되어 외국인 채권투자자금 유입이...,"[국제/NNG;금융시장/NNG;불안/NNG;확대/NNG, 원화/NNG;환율/NNG;..."
19,2010.08.12,금융통화위원회 의사록(2010년도 제16차)(2010.8.12),일부 위원은 연동대출 취급확대가COFIX｢ ｣통화정책에 미치는 영향으로 금리파급경로...,"[개인사업자/NNG;대출/NNG;늘/VV, 단기/NNG;금리/NNG;상승/NNG, ..."
25,2010.11.16,금융통화위원회 의사록(2010년도 제22차)(2010.11.16),일부 위원은 기준금리 인상이 내외금·｢ ｣리차 확대에 따른 재정거래 유인 증대를 초...,"[경기/NNG;회복/NNG;둔화/NNG;우려/NNG, 자본/NNG;유입/NNG;증가..."
26,2010.12.09,금융통화위원회 의사록(2010년도 제24차)(2010.12.9),일부 위원은 월중 기준금리 인상에도 불11｢ ｣구하고 국고채 년물 금리가 사상 최저...,"[국고채/NNG;발행/NNG;증가/NNG, 금리/NNG;인상/NNG]"
...,...,...,...,...
290,2023.04.11,금융통화위원회 의사록(2023년도 제7차)(2023.4.11),"일부 위원은 신협기구(상호금융, 신협, 새마을금고)의 부동산 관련 대출 비중이 과거...","[가격/NNG;하락/NNG;경상/NNG;수지/NNG;적자/NNG, 부동산/NNG;p..."
291,2024.05.23,금융통화위원회 의사록(2024년도 제10차)(2024.5.23),일부 위원은 유로지역 등 주요국이 미국에 앞서 선제적으로 정책금리를 인하할 가능성이...,"[글로벌/NNG;금융시장/NNG;위험/NNG;선호/NNG, 은행/NNG;대출/NNG..."
292,2024.01.11,금융통화위원회 의사록(2024년도 제1차)(2024.1.11),일부 위원은 국내외 금융시장이 지난 통화정책방향 결정회의 이후 대체로 완화적이며 안...,"[대출/NNG;가계/NNG;대출/NNG;증가/NNG, 가계/NNG;대출/NNG;관리..."
295,2024.02.22,금융통화위원회 의사록(2024년도 제4차)(2024.2.22),일부 위원은 최근 금융·외환시장은 대체로 안정적인 흐름을 이어가고 있으며지난 통화정...,"[기업/NNG;대출/NNG;연체율/NNG;상승/NNG, 글로벌/NNG;유동성/NNG..."


In [5]:
# DF 병합
call_m_df = pd.merge(df, call_df, left_on = "date", right_on = "date", how = "inner")
call_m_df

,date,title,contents,ex_ngrams,Unnamed: 0,call_rate,date_1m_bf,call_rate_1m_bf,Labeling
0,2010.05.12,금융통화위원회 의사록(2010년도 제10차)(2010.5.12),일부 위원은 남유럽사태 이후 각국이·｢ ｣금융기관에 대한 미시 거시 건전성감독의 중...,"[외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG, 장단기/NNG;금리/NN...",803,2.010,2010.04.12,2.010,Same
1,2010.07.09,금융통화위원회 의사록(2010년도 제14차)(2010.7.9),일부 위원은 금리 인상시 차익거래 유·｢ ｣인이 확대되어 외국인 채권투자자금 유입이...,"[국제/NNG;금융시장/NNG;불안/NNG;확대/NNG, 원화/NNG;환율/NNG;...",861,2.260,2010.06.09,1.970,Up
2,2010.08.12,금융통화위원회 의사록(2010년도 제16차)(2010.8.12),일부 위원은 연동대출 취급확대가COFIX｢ ｣통화정책에 미치는 영향으로 금리파급경로...,"[개인사업자/NNG;대출/NNG;늘/VV, 단기/NNG;금리/NNG;상승/NNG, ...",895,2.270,2010.07.12,2.270,Same
3,2010.11.16,금융통화위원회 의사록(2010년도 제22차)(2010.11.16),일부 위원은 기준금리 인상이 내외금·｢ ｣리차 확대에 따른 재정거래 유인 증대를 초...,"[경기/NNG;회복/NNG;둔화/NNG;우려/NNG, 자본/NNG;유입/NNG;증가...",991,2.490,2010.10.16,2.270,Up
4,2010.12.09,금융통화위원회 의사록(2010년도 제24차)(2010.12.9),일부 위원은 월중 기준금리 인상에도 불11｢ ｣구하고 국고채 년물 금리가 사상 최저...,"[국고채/NNG;발행/NNG;증가/NNG, 금리/NNG;인상/NNG]",1014,2.510,2010.11.09,2.270,Up
...,...,...,...,...,...,...,...,...,...
122,2023.04.11,금융통화위원회 의사록(2023년도 제7차)(2023.4.11),"일부 위원은 신협기구(상호금융, 신협, 새마을금고)의 부동산 관련 대출 비중이 과거...","[가격/NNG;하락/NNG;경상/NNG;수지/NNG;적자/NNG, 부동산/NNG;p...",5520,3.452,2023.03.11,3.429,Up
123,2024.05.23,금융통화위원회 의사록(2024년도 제10차)(2024.5.23),일부 위원은 유로지역 등 주요국이 미국에 앞서 선제적으로 정책금리를 인하할 가능성이...,"[글로벌/NNG;금융시장/NNG;위험/NNG;선호/NNG, 은행/NNG;대출/NNG...",5928,3.556,2024.04.23,3.532,Up
124,2024.01.11,금융통화위원회 의사록(2024년도 제1차)(2024.1.11),일부 위원은 국내외 금융시장이 지난 통화정책방향 결정회의 이후 대체로 완화적이며 안...,"[대출/NNG;가계/NNG;대출/NNG;증가/NNG, 가계/NNG;대출/NNG;관리...",5795,3.393,2023.12.11,3.712,Down
125,2024.02.22,금융통화위원회 의사록(2024년도 제4차)(2024.2.22),일부 위원은 최근 금융·외환시장은 대체로 안정적인 흐름을 이어가고 있으며지난 통화정...,"[기업/NNG;대출/NNG;연체율/NNG;상승/NNG, 글로벌/NNG;유동성/NNG...",5837,3.659,2024.01.22,3.523,Up


In [6]:
call_m_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             127 non-null    object 
 1   title            127 non-null    object 
 2   contents         127 non-null    object 
 3   ex_ngrams        127 non-null    object 
 4   Unnamed: 0       127 non-null    int64  
 5   call_rate        127 non-null    float64
 6   date_1m_bf       127 non-null    object 
 7   call_rate_1m_bf  127 non-null    float64
 8   Labeling         127 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 9.1+ KB


In [7]:
# 품사 필터링 : 명사(NNG), 형용사(VA, VAX), 부사(MAG), 동사(VA), 부정(MAG)
def filter_pos(ngrams) :
    pos = ["NNG", "VA", "VAX", "MAG", "VV"]
    filtered_ngrams = []
    for ngram in ngrams :
        if ";" in ngram :
            filtered_terms = []
            terms = ngram.split(';')
            for term in terms :
                word, tag = term.split('/')
                if tag in pos :
                    filtered_terms.append(term)
            filtered_ngrams.append(';'.join(filtered_terms))
        else :
            word, tag = ngram.split('/')
            if tag in pos :
                filtered_ngrams.append(term)
    return filtered_ngrams
    
# 문자 길이 한 글자 필터링
def filter_len(ngrams) :
    return [ngram for ngram in ngrams if len(ngram.split('/')[0]) > 1]

# ex_ngrams 컬럼 필터링하여 ngrams 컬럼에 저장
call_m_df["ngrams"] = call_m_df["ex_ngrams"].apply(filter_pos).apply(filter_len)

In [8]:
call_m_df = call_m_df.reindex(columns=["date", "title", "contents", "ngrams", "Labeling"])
call_m_df

,date,title,contents,ngrams,Labeling
0,2010.05.12,금융통화위원회 의사록(2010년도 제10차)(2010.5.12),일부 위원은 남유럽사태 이후 각국이·｢ ｣금융기관에 대한 미시 거시 건전성감독의 중...,"[외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG, 장단기/NNG;금리/NN...",Same
1,2010.07.09,금융통화위원회 의사록(2010년도 제14차)(2010.7.9),일부 위원은 금리 인상시 차익거래 유·｢ ｣인이 확대되어 외국인 채권투자자금 유입이...,"[국제/NNG;금융시장/NNG;불안/NNG;확대/NNG, 원화/NNG;환율/NNG;...",Up
2,2010.08.12,금융통화위원회 의사록(2010년도 제16차)(2010.8.12),일부 위원은 연동대출 취급확대가COFIX｢ ｣통화정책에 미치는 영향으로 금리파급경로...,"[개인사업자/NNG;대출/NNG;늘/VV, 단기/NNG;금리/NNG;상승/NNG, ...",Same
3,2010.11.16,금융통화위원회 의사록(2010년도 제22차)(2010.11.16),일부 위원은 기준금리 인상이 내외금·｢ ｣리차 확대에 따른 재정거래 유인 증대를 초...,"[경기/NNG;회복/NNG;둔화/NNG;우려/NNG, 자본/NNG;유입/NNG;증가...",Up
4,2010.12.09,금융통화위원회 의사록(2010년도 제24차)(2010.12.9),일부 위원은 월중 기준금리 인상에도 불11｢ ｣구하고 국고채 년물 금리가 사상 최저...,"[국고채/NNG;발행/NNG;증가/NNG, 금리/NNG;인상/NNG]",Up
...,...,...,...,...,...
122,2023.04.11,금융통화위원회 의사록(2023년도 제7차)(2023.4.11),"일부 위원은 신협기구(상호금융, 신협, 새마을금고)의 부동산 관련 대출 비중이 과거...","[가격/NNG;하락/NNG;경상/NNG;수지/NNG;적자/NNG, 부동산/NNG;p...",Up
123,2024.05.23,금융통화위원회 의사록(2024년도 제10차)(2024.5.23),일부 위원은 유로지역 등 주요국이 미국에 앞서 선제적으로 정책금리를 인하할 가능성이...,"[글로벌/NNG;금융시장/NNG;위험/NNG;선호/NNG, 은행/NNG;대출/NNG...",Up
124,2024.01.11,금융통화위원회 의사록(2024년도 제1차)(2024.1.11),일부 위원은 국내외 금융시장이 지난 통화정책방향 결정회의 이후 대체로 완화적이며 안...,"[대출/NNG;가계/NNG;대출/NNG;증가/NNG, 가계/NNG;대출/NNG;관리...",Down
125,2024.02.22,금융통화위원회 의사록(2024년도 제4차)(2024.2.22),일부 위원은 최근 금융·외환시장은 대체로 안정적인 흐름을 이어가고 있으며지난 통화정...,"[기업/NNG;대출/NNG;연체율/NNG;상승/NNG, 글로벌/NNG;유동성/NNG...",Up


In [9]:
# 의사록 전처리 결과물 다운로드
call_m_df.to_csv("./data/minutes_ngrams.csv", encoding = "utf-8")

In [10]:
call_m_df["ngrams"][0]

['외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG',
 '장단기/NNG;금리/NNG;하락/NNG',
 '둔화/NNG;물가/NNG;하락/NNG',
 '자산/NNG;가격/NNG;거품/NNG',
 '경제/NNG;활력/NNG;높/VV',
 '디플레이션/NNG;악순환/NNG',
 '외환보유액/NNG;증대/NNG',
 '금융시장/NNG;불안/NNG',
 '긍정적/VAX;효과/NNG',
 '불안/NNG;부작용/NNG',
 '고유동성/NNG;떨어/VV',
 '부채/NNG;축소/NNG',
 '성장/NNG;둔화/NNG',
 '소득/NNG;감소/NNG',
 '부채/NNG;부담/NNG',
 '흑자/NNG',
 '위험/NNG;빠지/VV',
 '위기/NNG;쉽/VA']

## Unique Value 추출

In [11]:
n_list = call_m_df["ngrams"].values.tolist()
n_list

[['외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG',
  '장단기/NNG;금리/NNG;하락/NNG',
  '둔화/NNG;물가/NNG;하락/NNG',
  '자산/NNG;가격/NNG;거품/NNG',
  '경제/NNG;활력/NNG;높/VV',
  '디플레이션/NNG;악순환/NNG',
  '외환보유액/NNG;증대/NNG',
  '금융시장/NNG;불안/NNG',
  '긍정적/VAX;효과/NNG',
  '불안/NNG;부작용/NNG',
  '고유동성/NNG;떨어/VV',
  '부채/NNG;축소/NNG',
  '성장/NNG;둔화/NNG',
  '소득/NNG;감소/NNG',
  '부채/NNG;부담/NNG',
  '흑자/NNG',
  '위험/NNG;빠지/VV',
  '위기/NNG;쉽/VA'],
 ['국제/NNG;금융시장/NNG;불안/NNG;확대/NNG',
  '원화/NNG;환율/NNG;변동성/NNG;확대/NNG',
  '경제/NNG;성장/NNG;잠재력/NNG;확충/NNG',
  '외환시장/NNG;변동성/NNG;확대/NNG',
  '경제/NNG;부담/NNG;주/VV',
  '투자자금/NNG;유입/NNG;증가/NNG',
  '은행/NNG;대출태도/NNG;강화/NNG',
  '중소기업/NNG;대출/NNG;위험/NNG',
  '절상/NNG;경제/NNG;악영향/NNG',
  '금리/NNG;변동성/NNG;확대/NNG',
  '어음/NNG;부도율/NNG;상승/NNG',
  '주택/NNG;경기/NNG;부진/NNG',
  '하락/NNG;경기/NNG;회복/NNG',
  '물가/NNG;상승/NNG;압력/NNG',
  '은행/NNG;금리/NNG;인상/NNG',
  '자금사정/NNG;악화/NNG',
  '투자심리/NNG;약화/NNG',
  '구조조정/NNG;미흡/NNG',
  '주택시장/NNG;부진/NNG',
  '주택가격/NNG;상승/NNG',
  '경제/NNG;긍정적/VAX',
  '자금/NNG;호전/NNG',
  '경기/NNG;상승/NNG',
  '변동성/NNG;줄/VV'],
 ['개

In [12]:
# case 1
# 2차원 리스트를 word/tag 형태로 쪼개어 1차원 리스트로 변환하는 함수
# list.extend(iterable) : 리스트 끝에 iterable의 모든 항목을 추가 확장
def flatten_and_split(ng_list) :
    flat_list = []
    for sublist in ng_list :
        for i in sublist :
            terms = i.split(';')
            flat_list.extend(terms)
    return flat_list

# case 2
# 2차원 함수를 1차원으로만 flatten 하는 함수
def flatten_list(ng_list) :
    flat_list = []
    for sublist in ng_list:
        for i in sublist:
            flat_list.append(i)
    return flat_list

In [13]:
# case 1
result = flatten_and_split(n_list)
len(result), result

(16865,
 ['외환보유액/NNG',
  '경상/NNG',
  '수지/NNG',
  '흑자/NNG',
  '장단기/NNG',
  '금리/NNG',
  '하락/NNG',
  '둔화/NNG',
  '물가/NNG',
  '하락/NNG',
  '자산/NNG',
  '가격/NNG',
  '거품/NNG',
  '경제/NNG',
  '활력/NNG',
  '높/VV',
  '디플레이션/NNG',
  '악순환/NNG',
  '외환보유액/NNG',
  '증대/NNG',
  '금융시장/NNG',
  '불안/NNG',
  '긍정적/VAX',
  '효과/NNG',
  '불안/NNG',
  '부작용/NNG',
  '고유동성/NNG',
  '떨어/VV',
  '부채/NNG',
  '축소/NNG',
  '성장/NNG',
  '둔화/NNG',
  '소득/NNG',
  '감소/NNG',
  '부채/NNG',
  '부담/NNG',
  '흑자/NNG',
  '위험/NNG',
  '빠지/VV',
  '위기/NNG',
  '쉽/VA',
  '국제/NNG',
  '금융시장/NNG',
  '불안/NNG',
  '확대/NNG',
  '원화/NNG',
  '환율/NNG',
  '변동성/NNG',
  '확대/NNG',
  '경제/NNG',
  '성장/NNG',
  '잠재력/NNG',
  '확충/NNG',
  '외환시장/NNG',
  '변동성/NNG',
  '확대/NNG',
  '경제/NNG',
  '부담/NNG',
  '주/VV',
  '투자자금/NNG',
  '유입/NNG',
  '증가/NNG',
  '은행/NNG',
  '대출태도/NNG',
  '강화/NNG',
  '중소기업/NNG',
  '대출/NNG',
  '위험/NNG',
  '절상/NNG',
  '경제/NNG',
  '악영향/NNG',
  '금리/NNG',
  '변동성/NNG',
  '확대/NNG',
  '어음/NNG',
  '부도율/NNG',
  '상승/NNG',
  '주택/NNG',
  '경기/NNG',
  '부진/NNG',
  '하락/N

In [14]:
# case 1
set_lst = set(result)
len(set_lst), set_lst

(740,
 {'bis/NNG',
  'cd/NNG',
  'cds/NNG',
  'cp/NNG',
  'crs/NNG',
  'ecb/NNG',
  'fed/NNG',
  'ff/NNG',
  'gdp/NNG',
  'ism/NNG',
  'mmf/NNG',
  'oecd/NNG',
  'opec/NNG',
  'pf/NNG',
  'rp/NNG',
  'ted스프레드/NNG',
  '가격/NNG',
  '가격변수/NNG',
  '가계/NNG',
  '가능성/NNG',
  '가속/NNG',
  '가처분/NNG',
  '감독/NNG',
  '감산/NNG',
  '감소/NNG',
  '감속/NNG',
  '감축/NNG',
  '강도/NNG',
  '강등/NNG',
  '강력/NNG',
  '강세/NNG',
  '강하/VA',
  '강화/NNG',
  '개방/NNG',
  '개선/NNG',
  '개인/NNG',
  '개인사업자/NNG',
  '갭/NNG',
  '거래/NNG',
  '거시/NNG',
  '거시정책/NNG',
  '거의/MAG',
  '거주자/NNG',
  '거품/NNG',
  '건설/NNG',
  '건전성/NNG',
  '견실/NNG',
  '견인/NNG',
  '견조/NNG',
  '결정/NNG',
  '경감/NNG',
  '경계/NNG',
  '경고/NNG',
  '경기/NNG',
  '경상/NNG',
  '경색/NNG',
  '경영/NNG',
  '경쟁력/NNG',
  '경제/NNG',
  '경제주체/NNG',
  '경제활동참가율/NNG',
  '경직/NNG',
  '경착륙/NNG',
  '경향/NNG',
  '고금리/NNG',
  '고물가/NNG',
  '고변동성/NNG',
  '고용/NNG',
  '고용시장/NNG',
  '고용자수/NNG',
  '고유동성/NNG',
  '고점/NNG',
  '고정/NNG',
  '고평가/NNG',
  '공급/NNG',
  '공기업/NNG',
  '공식적/VAX',
  '과도/NNG',
  '과열/NNG'

In [15]:
# case 2
result2 = flatten_list(n_list)
len(result2), result2

(6422,
 ['외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG',
  '장단기/NNG;금리/NNG;하락/NNG',
  '둔화/NNG;물가/NNG;하락/NNG',
  '자산/NNG;가격/NNG;거품/NNG',
  '경제/NNG;활력/NNG;높/VV',
  '디플레이션/NNG;악순환/NNG',
  '외환보유액/NNG;증대/NNG',
  '금융시장/NNG;불안/NNG',
  '긍정적/VAX;효과/NNG',
  '불안/NNG;부작용/NNG',
  '고유동성/NNG;떨어/VV',
  '부채/NNG;축소/NNG',
  '성장/NNG;둔화/NNG',
  '소득/NNG;감소/NNG',
  '부채/NNG;부담/NNG',
  '흑자/NNG',
  '위험/NNG;빠지/VV',
  '위기/NNG;쉽/VA',
  '국제/NNG;금융시장/NNG;불안/NNG;확대/NNG',
  '원화/NNG;환율/NNG;변동성/NNG;확대/NNG',
  '경제/NNG;성장/NNG;잠재력/NNG;확충/NNG',
  '외환시장/NNG;변동성/NNG;확대/NNG',
  '경제/NNG;부담/NNG;주/VV',
  '투자자금/NNG;유입/NNG;증가/NNG',
  '은행/NNG;대출태도/NNG;강화/NNG',
  '중소기업/NNG;대출/NNG;위험/NNG',
  '절상/NNG;경제/NNG;악영향/NNG',
  '금리/NNG;변동성/NNG;확대/NNG',
  '어음/NNG;부도율/NNG;상승/NNG',
  '주택/NNG;경기/NNG;부진/NNG',
  '하락/NNG;경기/NNG;회복/NNG',
  '물가/NNG;상승/NNG;압력/NNG',
  '은행/NNG;금리/NNG;인상/NNG',
  '자금사정/NNG;악화/NNG',
  '투자심리/NNG;약화/NNG',
  '구조조정/NNG;미흡/NNG',
  '주택시장/NNG;부진/NNG',
  '주택가격/NNG;상승/NNG',
  '경제/NNG;긍정적/VAX',
  '자금/NNG;호전/NNG',
  '경기/NNG;상승/NNG',
  '변동성/NNG;줄/VV',

In [16]:
# case 2
set_lst2 = set(result2)
len(set_lst2), set_lst2

(3510,
 {'기대수익률/NNG;낮/VA',
  '가격/NNG;오르/VV;상승/NNG',
  '수주/NNG;증가/NNG',
  '통화정책/NNG;대응/NNG;어렵/VA',
  '유로존/NNG;정치/NNG;불안/NNG',
  '불안/NNG;완화/NNG',
  '수출/NNG;비중/NNG;확대/NNG',
  '구조개혁/NNG;지연/NNG',
  '변동성/NNG;하락/NNG',
  '불균형/NNG;완화/NNG',
  '대외/NNG;안정/NNG',
  '기업/NNG;신용등급/NNG;하락/NNG',
  '자산/NNG;투자/NNG;증가/NNG',
  '위험/NNG;크/VV',
  '선진국/NNG;금리/NNG;상승/NNG',
  '금리/NNG;변동성/NNG;확대/NNG;우려/NNG',
  '호조/NNG;긍정적/VAX',
  '안전자산/NNG;선호/NNG',
  '캐리/NNG;수요/NNG;늘/VV',
  '디플레이션/NNG;우려/NNG',
  '산업/NNG;활력/NNG',
  '국고채/NNG;발행/NNG;확대/NNG',
  '외국인/NNG;증권/NNG;투자/NNG;확대/NNG',
  '수지/NNG;개선/NNG',
  '상환능력/NNG;취약/NNG',
  '건전성/NNG;안정/NNG',
  '신용등급/NNG;하락/NNG;우려/NNG',
  '외국인/NNG;증권/NNG;투자자금/NNG;유입/NNG;확대/NNG',
  '주택담보대출/NNG;감소/NNG',
  '신흥국/NNG;경기/NNG;둔화/NNG',
  '크레딧/NNG;스프레드/NNG;확대/NNG',
  '외국인/NNG;투자자금/NNG;유입/NNG;확대/NNG',
  '영업이익/NNG;감소/NNG',
  '대출/NNG;수요/NNG;증가/NNG',
  '주택/NNG;증가/NNG',
  '부실/NNG;확대/NNG',
  '비용/NNG;부담/NNG;늘/VV',
  '채권/NNG;수요/NNG;늘/VV',
  '신용등급/NNG;전망/NNG;하향/NNG',
  '회사채/NNG;시장/NNG;신용위험/NNG',
  '설비/NNG;투자/NN

In [17]:
# con_ngrams 컬럼의 각 요소들을 list -> str로 변환
call_m_df["con_ngrams"] = None
for i in range(len(call_m_df["ngrams"])) :
    call_m_df["con_ngrams"][i] = ", ".join(call_m_df["ngrams"][i])
call_m_df

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9728\965417701.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  call_m_df["con_ngrams"][i] = ", ".join(call_m_df["ngrams"][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel

,date,title,contents,ngrams,Labeling,con_ngrams
0,2010.05.12,금융통화위원회 의사록(2010년도 제10차)(2010.5.12),일부 위원은 남유럽사태 이후 각국이·｢ ｣금융기관에 대한 미시 거시 건전성감독의 중...,"[외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG, 장단기/NNG;금리/NN...",Same,"외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG, 장단기/NNG;금리/NNG..."
1,2010.07.09,금융통화위원회 의사록(2010년도 제14차)(2010.7.9),일부 위원은 금리 인상시 차익거래 유·｢ ｣인이 확대되어 외국인 채권투자자금 유입이...,"[국제/NNG;금융시장/NNG;불안/NNG;확대/NNG, 원화/NNG;환율/NNG;...",Up,"국제/NNG;금융시장/NNG;불안/NNG;확대/NNG, 원화/NNG;환율/NNG;변..."
2,2010.08.12,금융통화위원회 의사록(2010년도 제16차)(2010.8.12),일부 위원은 연동대출 취급확대가COFIX｢ ｣통화정책에 미치는 영향으로 금리파급경로...,"[개인사업자/NNG;대출/NNG;늘/VV, 단기/NNG;금리/NNG;상승/NNG, ...",Same,"개인사업자/NNG;대출/NNG;늘/VV, 단기/NNG;금리/NNG;상승/NNG, 자..."
3,2010.11.16,금융통화위원회 의사록(2010년도 제22차)(2010.11.16),일부 위원은 기준금리 인상이 내외금·｢ ｣리차 확대에 따른 재정거래 유인 증대를 초...,"[경기/NNG;회복/NNG;둔화/NNG;우려/NNG, 자본/NNG;유입/NNG;증가...",Up,"경기/NNG;회복/NNG;둔화/NNG;우려/NNG, 자본/NNG;유입/NNG;증가/..."
4,2010.12.09,금융통화위원회 의사록(2010년도 제24차)(2010.12.9),일부 위원은 월중 기준금리 인상에도 불11｢ ｣구하고 국고채 년물 금리가 사상 최저...,"[국고채/NNG;발행/NNG;증가/NNG, 금리/NNG;인상/NNG]",Up,"국고채/NNG;발행/NNG;증가/NNG, 금리/NNG;인상/NNG"
...,...,...,...,...,...,...
122,2023.04.11,금융통화위원회 의사록(2023년도 제7차)(2023.4.11),"일부 위원은 신협기구(상호금융, 신협, 새마을금고)의 부동산 관련 대출 비중이 과거...","[가격/NNG;하락/NNG;경상/NNG;수지/NNG;적자/NNG, 부동산/NNG;p...",Up,"가격/NNG;하락/NNG;경상/NNG;수지/NNG;적자/NNG, 부동산/NNG;pf..."
123,2024.05.23,금융통화위원회 의사록(2024년도 제10차)(2024.5.23),일부 위원은 유로지역 등 주요국이 미국에 앞서 선제적으로 정책금리를 인하할 가능성이...,"[글로벌/NNG;금융시장/NNG;위험/NNG;선호/NNG, 은행/NNG;대출/NNG...",Up,"글로벌/NNG;금융시장/NNG;위험/NNG;선호/NNG, 은행/NNG;대출/NNG;..."
124,2024.01.11,금융통화위원회 의사록(2024년도 제1차)(2024.1.11),일부 위원은 국내외 금융시장이 지난 통화정책방향 결정회의 이후 대체로 완화적이며 안...,"[대출/NNG;가계/NNG;대출/NNG;증가/NNG, 가계/NNG;대출/NNG;관리...",Down,"대출/NNG;가계/NNG;대출/NNG;증가/NNG, 가계/NNG;대출/NNG;관리/..."
125,2024.02.22,금융통화위원회 의사록(2024년도 제4차)(2024.2.22),일부 위원은 최근 금융·외환시장은 대체로 안정적인 흐름을 이어가고 있으며지난 통화정...,"[기업/NNG;대출/NNG;연체율/NNG;상승/NNG, 글로벌/NNG;유동성/NNG...",Up,"기업/NNG;대출/NNG;연체율/NNG;상승/NNG, 글로벌/NNG;유동성/NNG;..."


In [18]:
# 필요한 컬럼들로 reindex
result_df = call_m_df.reindex(columns=["date", "title", "con_ngrams", "Labeling"])
result_df = result_df.rename(columns={"con_ngrams": "ngrams"})
result_df

,date,title,ngrams,Labeling
0,2010.05.12,금융통화위원회 의사록(2010년도 제10차)(2010.5.12),"외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG, 장단기/NNG;금리/NNG...",Same
1,2010.07.09,금융통화위원회 의사록(2010년도 제14차)(2010.7.9),"국제/NNG;금융시장/NNG;불안/NNG;확대/NNG, 원화/NNG;환율/NNG;변...",Up
2,2010.08.12,금융통화위원회 의사록(2010년도 제16차)(2010.8.12),"개인사업자/NNG;대출/NNG;늘/VV, 단기/NNG;금리/NNG;상승/NNG, 자...",Same
3,2010.11.16,금융통화위원회 의사록(2010년도 제22차)(2010.11.16),"경기/NNG;회복/NNG;둔화/NNG;우려/NNG, 자본/NNG;유입/NNG;증가/...",Up
4,2010.12.09,금융통화위원회 의사록(2010년도 제24차)(2010.12.9),"국고채/NNG;발행/NNG;증가/NNG, 금리/NNG;인상/NNG",Up
...,...,...,...,...
122,2023.04.11,금융통화위원회 의사록(2023년도 제7차)(2023.4.11),"가격/NNG;하락/NNG;경상/NNG;수지/NNG;적자/NNG, 부동산/NNG;pf...",Up
123,2024.05.23,금융통화위원회 의사록(2024년도 제10차)(2024.5.23),"글로벌/NNG;금융시장/NNG;위험/NNG;선호/NNG, 은행/NNG;대출/NNG;...",Up
124,2024.01.11,금융통화위원회 의사록(2024년도 제1차)(2024.1.11),"대출/NNG;가계/NNG;대출/NNG;증가/NNG, 가계/NNG;대출/NNG;관리/...",Down
125,2024.02.22,금융통화위원회 의사록(2024년도 제4차)(2024.2.22),"기업/NNG;대출/NNG;연체율/NNG;상승/NNG, 글로벌/NNG;유동성/NNG;...",Up


In [19]:
# up, down 카운트 할 데이터프레임 생성하는 함수
def create_df(set_list) :
    data = {
        "word" : list(set_list),
        "up" : [0] * len(set_list),
        "down" : [0] * len(set_list),
        "total" : [0] * len(set_list)
    }
    df = pd.DataFrame(data)
    return df
    
cnt_df = create_df(set_lst)
cnt_df

,word,up,down,total
0,앞당기/VV,0,0,0
1,다수/NNG,0,0,0
2,직거래시장/NNG,0,0,0
3,막/VV,0,0,0
4,성장성/NNG,0,0,0
...,...,...,...,...
735,역전/NNG,0,0,0
736,주춤/NNG,0,0,0
737,저조/NNG,0,0,0
738,금리갭/NNG,0,0,0


In [20]:
word_set = set(cnt_df["word"])

for idx, row in tqdm(result_df.iterrows()) :
    ngrams_list = row["ngrams"].split(", ")
    labeling = row["Labeling"]
    
    for ngram in ngrams_list:
        terms = ngram.split(";")  # ngram을 "word/tag" 형식으로 분할
        for term in terms:
            word_tag = term.strip()  # 공백 제거!
            
            if word_tag in word_set :
                idx_list = cnt_df[cnt_df["word"] == word_tag].index.tolist()  # word_tag가 일치하는 인덱스 리스트 가져오기
                if idx_list :
                    idx = idx_list[0]  # 첫 번째 인덱스 가져오기
                    if labeling == "Up" :
                        cnt_df.at[idx, "up"] += 1
                    elif labeling == "Down" :
                        cnt_df.at[idx, "down"] += 1
                    cnt_df.at[idx, "total"] += 1
cnt_df

127it [00:11, 10.73it/s]


,word,up,down,total
0,앞당기/VV,0,1,1
1,다수/NNG,0,1,1
2,직거래시장/NNG,0,1,2
3,막/VV,1,3,4
4,성장성/NNG,0,2,3
...,...,...,...,...
735,역전/NNG,1,5,6
736,주춤/NNG,0,1,1
737,저조/NNG,1,0,1
738,금리갭/NNG,18,18,44


In [21]:
pos = cnt_df["up"].sum()
neg = cnt_df["down"].sum()
total_sum = cnt_df["total"].sum()
print(pos, neg, total_sum)
# pos + neg + same = total, same은 데이터프레임에서 제외

6284 8279 16865
